## Purpose
This notebook is just meant to scrape the air quality dataset

In [12]:
import requests
POSTCODES = {"GN4", 
             "BX2", 
             "WA9", 
             "MY7", 
             "HR1", 
             "LB5", 
             "KC5", 
             "TH2", 
             "BQ5", 
             "TD5", 
             "CD1", 
             "RI1", 
             "GR7", 
             "ME2", 
             "ST3", 
             "HV1", 
             "BQ6", 
             "LW1", 
             "HG1", 
             "BX9", 
             "ME1", 
             "TD0", 
             "CR5", 
             "KT3", 
             "NB1", 
             "CT6", 
             "WM6", 
             "BX0", 
             "WAA", 
             "GN2", 
             "GB0", 
             "BT6", 
             "BG1", 
             "EA6", 
             "GR5", 
             "GR4", 
             "WM0", 
             "HV3", 
             "CT2", 
             "LB6", "KC2", "EN7", "RB4", "WAB", "CT8", "KC3", "EI1", "LW4", "EN5", "GB6", "KC7", "IS6", "CR8", "CD9", "RB7", "TH6", "EN4", "IM1", "LW3", "ST5", "CR9", "GR9", "CT3", "WA7", "MY1", "LH0", "BQ7", "ME7", "HI0", "SK6", "HG4", "HF4", "WM8", "KC4", "EI8", "KT4", "CD3", "RI2", "BG2", "BX1", "BL0", "GR8", "KC1", "EN1", "BT4", "HR2", "HK6", "LB4", "TH4", "WA8", "WA2", "BQ8", "ST6", "ST8", "IS2", "GN0", "LW2", "SK5", "GN3", "CR7", "ST4", "EA8", "CT4", "TH5", "BT5"}

In [2]:
old_size = len(POSTCODES)
code = next(iter(POSTCODES))
new_size = len(POSTCODES)
assert old_size == new_size

In [73]:
import string
import joblib

BASE_URL ="http://api.erg.kcl.ac.uk/AirQuality/Data/Site/SiteCode={code}/StartDate={start}-{start_month}-01-1200/EndDate={end}-{end_month}-01-1200/Json"
mem = joblib.Memory(cachedir="data")
def scrape_info(postcode, start, end, start_month, end_month):
    retries = 3
    page = requests.get(BASE_URL.format(start=start, end=end, code=postcode, start_month=start_month, end_month=end_month))
    if page.status_code == 400 and 'timeout' in page.text.lower():
        # just do a single retry
        page = requests.get(BASE_URL.format(start=start, 
                                            end=end, 
                                            code=postcode,
                                            start_month=start_month,
                                            end_month=end_month))
    
    page.raise_for_status()
    return page.json()

scrape_mem = mem.cache(scrape_info)

def get_info_for_code(code):
    scrape_mem(code, 2017, 2017, "01", "08")

In [74]:
p = joblib.Parallel(n_jobs=5)

In [76]:
[get_info_for_code(code) for code in POSTCODES]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]